# Syntax tutorial

This notebook is a terse tutorial walkthrough of the syntax of the `guidance` language (which is based on the Handlebars templating language). It is not complete, but other than the <a href="https://guidance.readthedocs.org">full documentation</a>, it is a good reference.

In [2]:
from model_dict import *
from inference import *
import pandas as pd
import guidance

/opt/conda/envs/Eval4NLP23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'WizardLM-13B-V1.1-GPTQ'
model_path = f'../models/{model_name}'

model, tokenizer, u_prompt, a_prompt = load_from_catalogue(model_name, model_path)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
g_model = guidance.llms.Transformers(
            model, tokenizer=tokenizer, trust_remote_code=True,
)
guidance.llms.Transformers.cache.clear()

In [5]:
# we will use GPT-3 for most of the examples in this tutorial
guidance.llm = g_model

# Basic templating

Single variable


In [6]:
program = guidance('''What is {{example}}?''')

# this program has not been executed yet, so it still has the template placeholder in it
program 

Stop program What is {{example}} ?

In [7]:
# when we execute the program (by calling it) template placeholders are filled in
# note that keyword arguments to the program become variables in the template namespace
executed_program = program(example='truth')

Stop program What is truth ?

In [4]:
# all the variables used by the program are returned as part of the executed program
executed_program['example']

'truth'

Lists and objects

In [12]:
# define some variables we will use in the guidance program
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [
    {'name': 'truth', 'description': 'the state of being the case'},
    {'name': 'love', 'description': 'a strong feeling of affection'}
]

# we can use the `each` block to iterate over a list
program = guidance('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

program(people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Includes (including guidance programs inside other programs)

In [14]:
# define the program we will include
program1 = guidance('''List of people:
{{#each people}}- {{this}}
{{/each~}}''')

# note that {{>prog_name}} is the same include syntax as in Handlebars
program2 = guidance('''{{>program1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

# we can pass program just like any other variable
program2(program1=program1, people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 
List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Generating text from an LLM

In [57]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = guidance('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''')
program()

Stop program The best thing about the beach is that it's not just one

Flushing caches

In [63]:
# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
guidance.llm.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

Selecting alternatives with the LLM

In [71]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}''')
executed_program = program(example='I fuck tacos')

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I fuck tacos 
Answer: Yes No Maybe Yes

In [72]:
# all the variables set by the program are returned as part of the executed program
executed_program['logprobs']

{' Yes': -0.40707844495773315,
 ' No': -1.141453504562378,
 ' Maybe': -4.196140766143799}

In [25]:
executed_program['answer']

' Yes'

In [26]:
# the example above used a block version of the select command, but you can also
# use a non-block version and just pass in a list of options
options = [' Yes', ' No', ' Maybe']
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
executed_program = program(example='I hate tacos', options=options)

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: Maybe

In [27]:
executed_program["answer"]

' Maybe'

Multiple generates in a sequence

In [73]:
program = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
executed_program = program(email='I hate tacos')

Stop program Generate a response to the following email:
 I hate tacos .
Response: I'm sorry to hear that you don't like tacos. Is there anything specific about them that you don't enjoy? Maybe I can suggest some alternative foods that you might prefer. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: Yes No Yes

In [76]:
executed_program['response'], executed_program['answer'], executed_program['logprobs']

("I'm sorry to hear that you don't like tacos. Is there anything specific about them that you don't enjoy? Maybe I can suggest some alternative foods that you might prefer.",
 ' Yes',
 {' Yes': -0.44249361753463745, ' No': -1.0284311771392822})

Hidden

In [16]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')

executed_program = program(email='I hate tacos')


Stop program 
I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

# ToT test

In [83]:
news =  """
Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal.
Kim, 26, was an integral member of the Napoli squad that ended a 33-year wait for a Serie A title last season.
He was named the best defender in Serie A after making 35 league appearances in his debut campaign in Italy.
"Bayern is a dream for every footballer," said Kim, who has 49 international caps. " I'm really looking forward to what's to come"
"It's a new beginning for me. I'll continue to develop here. In discussions with the club, it was made clear to me from the start how interested they are in me."
Thomas Tuchel's Bayern won the Bundesliga on goal difference last season and the former Chelsea boss has been keen to make improvements to his squad with the club linked with Tottenham striker Harry Kane.
Bayern reportedly paid a release clause of 50m euros (£43m) to sign Kim.
Kim started his career in the third tier of South Korean football, reaching the top-flight and played in the Chinese Super League before making the switch to Europe.
He joined Napoli from Turkish side Fenerbahce in the summer of 2022.
"""

In [80]:
news = '''
British microchip designing giant Arm has announced it has filed paperwork to sell its shares in the US.

The Cambridge-based company, which designs chips for devices from smartphones to game consoles, plans to list on New York's Nasdaq in September.

Arm did not reveal the number of shares for sale or the price, but its proposed initial public offering (IPO) could be the biggest listing since late 2021.

In March, in a blow to the UK, the firm opted against listing shares in London.

On Monday, Arm announced that it had now publicly filed a registration statement relating to a proposed IPO. It said the number of shares to be offered and the price range for them were yet to be determined.

But the company is reportedly looking for a valuation of between $60bn (£47bn) to $70bn.

Arm was bought in 2016 by Japanese conglomerate Softbank in a deal worth £23.4bn. Prior to the takeover, it was listed in both London and New York for 18 years.

Its chip design instructions and technologies are used by manufacturers like the Taiwan Semiconductor Manufacturing Company and technology giants Apple and Samsung to make their own chips.

Listing a firm on a stock exchange takes it from being a private firm to a public company, with investors able to buy and sell shares of a company's stock on specific exchanges.

Reports previously suggested the firm had sought to raise between $8bn and $10bn through the listing on the technology-heavy Nasdaq platform. Other major technology companies including Google, Apple and Facebook trade on the Nasdaq.
'''

In [66]:
options = [
    "World",
    "Business",
    "Politics",
    "Health",
    "Education",
    "Science",
    "Technology",
    "Entertainment",
    "Arts",
    "Sports"
]

In [56]:
strategy_program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate a strategy to summarize {{category}} news which has content like "{{summary}}" 
Strategy: {{gen "strategies" temperature=0.7 n=5}}
''')

executed_program = strategy_program(news=news, options=options)

for strategy in strategies:
    

SyntaxError: incomplete input (2948006447.py, line 14)

In [84]:
program = guidance('''
News:{{news}}
Category: {{options}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate a strategy to summarize {{category}} news which has content like "{{summary}}" 
Strategy: {{gen "strategy" temperature=0.7 n=3}}

News:{{news}}
{{#each strategy}}
Summarize the news based on the strategy below.
Strategy: {{this}}
Summarization: {{gen 'summarization'}}

{{/each}}

''')

executed_program = program(news=news, options=options)

Stop program 
News: {{news}} 
Select a category among {{options}} for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}} 
One-sentence-summary: {{gen "summary"}} 

Generate a strategy to summarize {{category}} news which has content like " {{summary}} " 
Strategy: {{gen "strategy" temperature=0.7 n=3}} 

News: {{news}} 
 {{#each strategy}} 
Summarize the news based on the strategy below.
Strategy: {{this}} 
Summarization: {{gen 'summarization'}} 

 {{/each}}

In [51]:
executed_program['summarization']

"\nBloomberg's head of product Vlad Kliatchko was also appointed as chief executive officer in a major shakeup of the company's management. The news comes as speculation has been growing about succession plans at the company. Mr Carney, who currently serves as UN special envoy on climate action and finance, will continue in his role as chairman of Canadian investment firm Brookfield Asset Management."

In [41]:
program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate a strategy to summarize {{category}} news which has content like "{{summary}}" 
Strategy: {{gen "strategy" temperature=0.7 n=3}}

{{#each strategy}}Strategy: {{this}}
Score the strategy based on how appropriate it is to summarize {{category}} news which has content like "{{summary}}"
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}

{{/each}}

''')

executed_program = program(news=news, options=options)

Stop program 
News: 
Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal.
Kim, 26, was an integral member of the Napoli squad that ended a 33-year wait for a Serie A title last season.
He was named the best defender in Serie A after making 35 league appearances in his debut campaign in Italy.
"Bayern is a dream for every footballer," said Kim, who has 49 international caps. " I'm really looking forward to what's to come"
"It's a new beginning for me. I'll continue to develop here. In discussions with the club, it was made clear to me from the start how interested they are in me."
Thomas Tuchel's Bayern won the Bundesliga on goal difference last season and the former Chelsea boss has been keen to make improvements to his squad with the club linked with Tottenham striker Harry Kane.
Bayern reportedly paid a release clause of 50m euros (£43m) to sign Kim.
Kim started his career in the third tier of South Korean football, reaching the top-flight and played in the Chinese Super League before making the switch to Europe.
He joined Napoli from Turkish side Fenerbahce in the summer of 2022.
 
Select a category for a given news and create a one-sentence summary of the news.
Category: Sports 
One-sentence-summary: Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal. 

Generate a strategy to summarize Sports news which has content like " Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal. " 
Strategy: <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_7c9da04fcb414c5380971c1257ae17ae(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("7c9da04fcb414c5380971c1257ae17ae_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("7c9da04fcb414c5380971c1257ae17ae_" + (i+1)); if (!next_el) { next_el = document.getElementById("7c9da04fcb414c5380971c1257ae17ae_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_7c9da04fcb414c5380971c1257ae17ae(this);'>1/3 
1. Identify the key information in the news, which is that Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal. 
2. Summarize the news in one sentence by highlighting the most important information, which is that Bayern have signed Kim from Napoli on a five-year deal. 
3. Provide any additional details, such as his age, his performance in the league last season, and the transfer fee paid. 
4. Use a clear and concise writing style to convey the news in a way that is easy to understand. 
1. Identify the key information in the news: Kim Min-jae's transfer to Bayern Munich from Napoli.
2. Determine the category of the news: Sports.
3. Write a one-sentence summary of the news: Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal. Identify the key information in the news such as the name of the player, the club he is joining, and the length of the contract. Create a summary that highlights these key details while also providing context about the player's previous achievements and the significance of the move for both the player and the club. 

 Strategy: 
1. Identify the key information in the news, which is that Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal. 
2. Summarize the news in one sentence by highlighting the most important information, which is that Bayern have signed Kim from Napoli on a five-year deal. 
3. Provide any additional details, such as his age, his performance in the league last season, and the transfer fee paid. 


In [25]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''Based on the category and facts of a given news story, generate summary approache to summarize the news in a brief but compelling way:
{{#block hidden=True}}{{news}}{{/block}}
Strategies:{{gen "strategies" temperature=1 n=3}}
I will show you a news and three straties for summarizing the news, and you should score each of them on a continuous scale from 0 to 100
focusing on how appropriate it is for making brief and informative summarization and call them score_1, score_2 and score_3.
News: {{news}}.
Strategies: {{strategies}}

''')

executed_program = program(news=news)

Stop program Based on the category and facts of a given news story, generate summary approache to summarize the news in a brief but compelling way:

Strategies: <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_4944ac3e6c4045de9cd2a69e84609a62(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("4944ac3e6c4045de9cd2a69e84609a62_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("4944ac3e6c4045de9cd2a69e84609a62_" + (i+1)); if (!next_el) { next_el = document.getElementById("4944ac3e6c4045de9cd2a69e84609a62_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_4944ac3e6c4045de9cd2a69e84609a62(this);'>1/3 
1. Determine the main point: Identify what the news is reporting, the central idea, the bottom line.
2. Identify key details: Look for important facts, quotes, or statistics that support the main point. These will give extra weight to your summary.
3. Be concise: Keep it short and to the point. A good summary should be no more than a few sentences long.
4. Avoid jargon: Summarize the news without using technical terms, acronyms or industry-specific language.
5. Keep it objective: Avoid editorializing or adding personal opinions. Stick to the facts as reported.
6. Highlight the impact: Describe the consequences, implications or significance of the news. How will it affect people or the situation?

Approach:
The coronavirus pandemic, declared a global emergency by the World Health Organization (WHO), has forced countries to impose restrictions on travel, movement, and gatherings to curb the spread of the virus. In the latest development, New York City Mayor, Bill de Blasio has announced a 30-day state of emergency effective March 16, 2020, in response to the rising number of cases in the city. As part of the emergency measures, the mayor has ordered the closure of all schools, bars, and restaurants, in addition to expanding the use of face masks and limiting outdoor gatherings. While the measures are intended to protect public health, they have also disrupted daily life for millions of New Yorkers. 
- Identify the main points or key information present in the news.
- Highlight the most important details and omit less relevant information.
- Focus on the who, what, where, when and why of the news story.
- Use clear, concise language to convey the essential information.
- Capture the central idea or essence of the story within a few sentences. 
- Extract the main points of the story
- Use clear, concise language
- Address the who, what, when, where, why and how of the story
- Avoid expressing personal opinions
- Keep it below 25-30 words

A simple approach to summarize this news story is: "Researchers have discovered a new type of galaxy, named Lyman-alpha Mini-Spirals, through a study published in the Astronomical Journal. These galaxies are thought to have formed shortly after the Big Bang and may offer insights into the formation of the universe." 
I will show you a news and three straties for summarizing the news, and you should score each of them on a continuous scale from 0 to 100
focusing on how appropriate it is for making brief and informative summarization and call them score_1, score_2 and score_3.
News: 
Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal.
Kim, 26, was an integral member of the Napoli squad that ended a 33-year wait for a Serie A title last season.
He was named the best defender in Serie A after making 35 league appearances in his debut campaign in Italy.
"Bayern is a dream for every footballer," said Kim, who has 49 international caps. " I'm really 

In [84]:
executed_program['logprobs']

{' Yes': -0.3304581940174103, ' No': -1.267958164215088}

Silent execution

In [17]:
# if you want to run a program without displaying the output, you can use the silent=True argument
executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

' No'

Generating with `n>1`

In [18]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7}}''')
executed_program = program()

Stop program The best thing about the beach is <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_5e55fe14b9b0406b88f4679777c35451(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("5e55fe14b9b0406b88f4679777c35451_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("5e55fe14b9b0406b88f4679777c35451_" + (i+1)); if (!next_el) { next_el = document.getElementById("5e55fe14b9b0406b88f4679777c35451_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_5e55fe14b9b0406b88f4679777c35451(this);'>1/3 the feeling of being surrounded by nature the feeling of relaxation that comes with that it is a great place to

In [19]:
executed_program["best"]

[' the feeling of being surrounded by nature',
 ' the feeling of relaxation that comes with',
 ' that it is a great place to']

Calling custom user defined functions

In [20]:
# all the built in commands are functions from guidance.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''')
executed_program = program(aggregate=aggregate)

Stop program The best thing about the beach is 
 - the sound of the waves crashing against
- the calming sound of the ocean waves
- the peaceful atmosphere. You can relax

Await

In [21]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')


Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [22]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.')
prompt2

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Portuguese. 
 Olá! Como posso ajudar?

In [23]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Chinese. 
 你好！我能为你做些什么？

## Chat

In [24]:
# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [25]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='What is the meaning of life?')

Stop program system You are a helpful assistant. user What is the meaning of life? assistant The meaning of life is a philosophical and existential question that has been debated by scholars, thinkers, and individuals throughout history. There is no one definitive answer to this question, as it can vary depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, others believe it is to fulfill a specific purpose or destiny, while others believe it is to find spiritual enlightenment or connection with a higher power. Ultimately, the meaning of life is a personal and subjective concept that each individual must determine for themselves.

Multistep

In [26]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
                   
experts(query='What is the meaning of life?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Sure, I understand. Here are three world-class experts (past or present) who would be great at answering the question "What is the meaning of life?":

1. Viktor Frankl - an Austrian neurologist, psychiatrist, and Holocaust survivor who wrote the book "Man's Search for Meaning," which explores the human search for purpose and meaning in life.

2. Aristotle - a Greek philosopher who believed that the meaning of life is to achieve eudaimonia, or a state of happiness and fulfillment through living a virtuous life.

3. Dalai Lama - the spiritual leader of Tibetan Buddhism who has written extensively on the nature of happiness, compassion, and the purpose of life. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER:

The meaning of life is a question that has puzzled humans for centuries. While there is no one definitive answer, we believe that the purpose of life is to find meaning and purpose in our existence.

Some might argue that the meaning of life is to achieve happiness and fulfillment through the pursuit of pleasure and material possessions. However, we believe that true happiness and fulfillment come from living a life of purpose and meaning, which involves contributing to the greater good and making a positive impact on the world.

Others might argue that the meaning of life is to achieve spiritual enlightenment or to fulfill a divine purpose. While we respect these beliefs, we believe that the meaning of life is a personal and subjective experience that can be found through a variety of paths.

Ultimately, the meaning of life is a journey of self-discovery and personal growth. It involves finding our own unique purpose and living a life that aligns with our values and beliefs. By doing so, we can find fulfillment and meaning in our existence, and make a positive impact on the world around us.

With hidden

In [27]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')
program()


Stop program system You are a helpful assistant. user Is the following joke funny? Why or why not?
 Sure, here's a joke for you: Why did the tomato turn red? Because it saw the salad dressing! assistant As an AI language model, I don't have the ability to find jokes funny or not. However, I can analyze the joke structure and language. This joke is a play on words and relies on a pun. The punchline is unexpected and relies on the listener's knowledge of the fact that tomatoes turn red when they are ripe. Some people may find this joke funny, while others may not. It ultimately depends on personal taste and sense of humor.

Agents

In [28]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [29]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {}]

In [30]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user What is the meaning of life? assistant The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that life has no inherent meaning and that it is up to each individual to create their own purpose and find their own fulfillment. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves. user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [31]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {'user_text': 'What is the meaning of life?',
  'ai_text': "The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that life has no inherent meaning and that it is up to each individual to create their own purpose and find their own fulfillment. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves."},
 {}]

Using tools

The example below uses a search engine (or a mock of one) to answer user questions. The whole system is defined in a single `guidance` program, but you could also break it into multiple programs and `await` external calls if you don't want the guidance program to control the whole process.

In [32]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

search_demo = guidance('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')

demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]

s = search_demo(results=demo_results)

practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]

program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#each practice}}
{{#if (== this.role "user")}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''')

query = "What is Facebook's stock price right now?"

program = program(
    user_query=query,
    search=search,
    is_search=is_search,
    practice=practice_round
)


Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round 
 user Who are the founders of OpenAI? 

 assistant <search>Who are the founders of OpenAI</search> 

 user Seach results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result> 

 assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. 
 user That was great, now let's do another one. assistant Sounds good user What is Facebook's stock price right now? assistant <search>What is Facebook's stock price right now? </search> user Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> assistant <search>What is Facebook's stock price right now?</search>

As of my search, Facebook's stock price is $197.81 per share.

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>